<a href="https://githubtocolab.com/Astolfo2332/HyM_prediction/blob/main/0.2_Limpieza_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Limpieza de datos

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
'chmod 600 /root/.kaggle/kaggle.json' #Para proteger la información

In [ ]:
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f articles.csv
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f customers.csv
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f transactions_train.csv
! mkdir Data
! unzip transactions_train.csv.zip -d Data
! unzip customers.csv.zip -d Data
! unzip articles.csv.zip -d Data


## 1. Generación de datos faltantes

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn import preprocessing as ps
from difflib import SequenceMatcher
import csv

En este caso vamos a generar 5% de datos faltantes en las columnas de interés, así que aleatoriamente se desorganizan los datasets antes de unirlos en uno y se eliminaran en parte de transacciones el 5% de artículos comprados, en clientes; la edad y finalmente en artículos; el tipo de articulo.

In [3]:
client=pd.read_csv("Data/customers.csv")

In [4]:
sells=pd.read_csv("Data/transactions_train.csv")

In [5]:
articles=pd.read_csv("Data/articles.csv")

In [6]:
def makeitrandom(client,sells,articles):
    client=client.sample(frac=1) #Desorganizamos las filas
    sells=sells.sample(frac=1)
    articles=articles.sample(frac=1)
    client=client.reset_index(drop=True) #Reiniciamos los index
    sells=sells.reset_index(drop=True)
    articles=articles.reset_index(drop=True)
    return client,sells,articles

In [7]:
client,sells,articles=makeitrandom(client,sells,articles)

In [8]:
f=0.05
n_client=int(len(client)*f)
n_sells=int(len(sells)*f)
n_articles=int(len(articles)*f)

In [9]:
client.loc[:n_client,"age"]=np.nan
sells.loc[:n_sells,"article_id"]=np.nan
articles.loc[:n_articles,"product_type_name"]=np.nan

In [10]:
client,sells,articles=makeitrandom(client,sells,articles)

In [11]:
def faltap(data):
    total      = data.isnull().sum().sort_values(ascending=False)
    percent    = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending=False)
    missing  = pd.concat([total,percent],axis=1,keys=["Total","Percent"])
    return missing

In [12]:
faltap(client)

,Total,Percent
Active,907576,66.150819
FN,895050,65.237831
age,83652,6.097173
fashion_news_frequency,16009,1.166854
club_member_status,6062,0.441843
customer_id,0,0.000000
postal_code,0,0.000000


In [13]:
faltap(sells)

,Total,Percent
article_id,1589417,5.000003
t_dat,0,0.000000
customer_id,0,0.000000
price,0,0.000000
sales_channel_id,0,0.000000


In [14]:
faltap(articles)

,Total,Percent
product_type_name,5278,5.000853
detail_desc,416,0.394156
perceived_colour_master_name,0,0.000000
garment_group_name,0,0.000000
garment_group_no,0,0.000000
section_name,0,0.000000
section_no,0,0.000000
index_group_name,0,0.000000
index_group_no,0,0.000000
index_name,0,0.000000


## 2. agrupación de datos y eliminación de variables

En este caso vamos a proceder a primero hacer una correlación con las compras y los artículos de cada cliente, para generar un dataframe, que nos resuma los artículos comprados Por cliente, el dataset al ser tan grande primero se le hará una reducción a las transacciones y una división entre train y test por fechas

In [15]:
a=len(sells)//6
sells=sells.loc[0:a]
sells.index=sells.customer_id.values
del(sells["sales_channel_id"])
sells.head()

,t_dat,customer_id,article_id,price
d33144f46a7f6f37f4daaa2b120d3ca43cf205f397775fb65e9a3e45c83e8b5d,2020-07-18,d33144f46a7f6f37f4daaa2b120d3ca43cf205f397775f...,875239001.0,0.033881
75383435a42ad27392999e608ac0201f9df146e6579fe6d4cf2fe4a4a7f40324,2019-07-25,75383435a42ad27392999e608ac0201f9df146e6579fe6...,783264001.0,0.095424
d96d6df270d0edb3fac25b4c2c3da2c9bf22a1652d06e23d9518c3d6d944a34b,2019-06-27,d96d6df270d0edb3fac25b4c2c3da2c9bf22a1652d06e2...,761712005.0,0.030492
aa80f0017561281e8beb6a404e57f8a8a0b61f06430ce0e2e6fd9d4436c7e3e1,2020-09-20,aa80f0017561281e8beb6a404e57f8a8a0b61f06430ce0...,834906012.0,0.016932
94cee0f70da39f7e7a9164e40c4448d2118c87289a3fe8365bc14e406cf64de2,2019-09-13,94cee0f70da39f7e7a9164e40c4448d2118c87289a3fe8...,788324001.0,0.044051


In [16]:
client.index=client.customer_id.values
del(client["postal_code"])
del(client["customer_id"])
client.head()

,FN,Active,club_member_status,fashion_news_frequency,age
b971e494732062eef926945a47bafb01785ab96f36d1fcd99ca0776ad4052d06,1.0,1.0,ACTIVE,Regularly,46.0
0a0aa7939630fa6dd49ce963078e8d19e1ec1603c77ffdb1694afe4083f16025,NaN,NaN,ACTIVE,NONE,NaN
30a210d92448256556595b6116cd1d8e831f68b4c59744168c03fb1ae283c759,NaN,NaN,ACTIVE,NONE,24.0
214f63dd15856099a80bbef91adfe5c549e4d50b6e09966dbfcbca24feb9806f,NaN,NaN,ACTIVE,NONE,57.0
bbff1987292c5d1e6b94645972032823e23f4835470405604705bf22797d5b8d,NaN,NaN,PRE-CREATE,NONE,27.0


In [17]:
sells=sells.join(client)
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023.0,0.035576,NaN,NaN,ACTIVE,NONE,49.0
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2019-11-22,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785186005.0,0.016932,NaN,NaN,ACTIVE,NONE,49.0
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,2019-07-05,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,701561001.0,0.059305,NaN,NaN,ACTIVE,NONE,25.0
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,2019-08-27,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,732842008.0,0.067780,NaN,NaN,ACTIVE,NONE,25.0
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,2019-05-07,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,723529001.0,0.025407,NaN,NaN,ACTIVE,NONE,25.0


In [18]:
sells=sells.reset_index(drop=True)

In [19]:
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age
0,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023.0,0.035576,NaN,NaN,ACTIVE,NONE,49.0
1,2019-11-22,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785186005.0,0.016932,NaN,NaN,ACTIVE,NONE,49.0
2,2019-07-05,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,701561001.0,0.059305,NaN,NaN,ACTIVE,NONE,25.0
3,2019-08-27,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,732842008.0,0.067780,NaN,NaN,ACTIVE,NONE,25.0
4,2019-05-07,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,723529001.0,0.025407,NaN,NaN,ACTIVE,NONE,25.0


## 2.1 Relleno de datos vacíos  

En este caso vamos a reemplazar los datos faltantes, en el caso de el articulo, fashion_news, con la moda, el FN y active con 0. Y age con el promedio.

In [20]:
sells["article_id"].fillna(sells.article_id.mode()[0], inplace=True)
sells["FN"].fillna(0, inplace=True)
sells["Active"].fillna(0, inplace=True)
sells["club_member_status"].fillna(sells.club_member_status.mode()[0], inplace=True)
sells["fashion_news_frequency"].fillna(sells.fashion_news_frequency.mode()[0], inplace=True)
sells["age"].fillna(int(sells.age.mean()), inplace=True)

In [21]:
sells.isnull().sum()

t_dat                     0
customer_id               0
article_id                0
price                     0
FN                        0
Active                    0
club_member_status        0
fashion_news_frequency    0
age                       0
dtype: int64

## 2.2 Filtrado de datos y concatenación final

Ahora vamos a filtrar los artículos para las columnas que consideramos afecten más la compra futura, como es el tipo de articulo, color, la sección a la que pertenece (ya que de ahí podríamos inferir el genero del cliente)

In [22]:
sells.index=sells.article_id.values
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age
176209023.0,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023.0,0.035576,0.0,0.0,ACTIVE,NONE,49.0
785186005.0,2019-11-22,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785186005.0,0.016932,0.0,0.0,ACTIVE,NONE,49.0
701561001.0,2019-07-05,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,701561001.0,0.059305,0.0,0.0,ACTIVE,NONE,25.0
732842008.0,2019-08-27,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,732842008.0,0.067780,0.0,0.0,ACTIVE,NONE,25.0
723529001.0,2019-05-07,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,723529001.0,0.025407,0.0,0.0,ACTIVE,NONE,25.0


In [23]:
articles=articles[["product_type_name","article_id","colour_group_name","index_group_name","section_name"]]
articles.index=articles.article_id.values
del(articles["article_id"])

In [24]:
sells=sells.join(articles)
sells=sells.reset_index(drop=True)
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
0,2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,48.0,Vest top,Black,Ladieswear,Womens Everyday Basics
1,2019-02-26,005b40b50b0884068e374dbc18dfb712f2a9282f12d489...,108775015.0,0.008458,1.0,1.0,ACTIVE,Regularly,51.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2,2019-02-26,005b40b50b0884068e374dbc18dfb712f2a9282f12d489...,108775015.0,0.008458,1.0,1.0,ACTIVE,Regularly,51.0,Vest top,Black,Ladieswear,Womens Everyday Basics
3,2018-12-06,0064dc87f5ef46052ff9cfce969f7f4fe6874e2b23417b...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,38.0,Vest top,Black,Ladieswear,Womens Everyday Basics
4,2019-01-23,00815f271d1ff0dfad69b84560db517aa7e553f01dea90...,108775015.0,0.008458,0.0,0.0,PRE-CREATE,NONE,36.0,Vest top,Black,Ladieswear,Womens Everyday Basics


In [25]:
sells.index=pd.to_datetime(sells.t_dat)
del(sells["t_dat"])
sells.head()

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
t_dat,,,,,,,,,,,,
2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,48.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2019-02-26,005b40b50b0884068e374dbc18dfb712f2a9282f12d489...,108775015.0,0.008458,1.0,1.0,ACTIVE,Regularly,51.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2019-02-26,005b40b50b0884068e374dbc18dfb712f2a9282f12d489...,108775015.0,0.008458,1.0,1.0,ACTIVE,Regularly,51.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2018-12-06,0064dc87f5ef46052ff9cfce969f7f4fe6874e2b23417b...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,38.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2019-01-23,00815f271d1ff0dfad69b84560db517aa7e553f01dea90...,108775015.0,0.008458,0.0,0.0,PRE-CREATE,NONE,36.0,Vest top,Black,Ladieswear,Womens Everyday Basics


In [26]:
sells.tail()

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
t_dat,,,,,,,,,,,,
2020-09-18,1ddbd4aee17fb44361f192c21fec9410e292f5472579b0...,956217002.0,0.059305,0.0,0.0,ACTIVE,NONE,55.0,Dress,Black,Ladieswear,Womens Trend
2020-09-15,38155dba0255d0b6f8884b36dd05094d62c71ca9c10311...,956217002.0,0.059305,1.0,1.0,ACTIVE,Regularly,24.0,Dress,Black,Ladieswear,Womens Trend
2020-09-13,6b05a969d676cf290cd44fc55ab3067d8406fb3e3bbc47...,956217002.0,0.059305,0.0,0.0,ACTIVE,NONE,50.0,Dress,Black,Ladieswear,Womens Trend
2020-09-09,a7b78afbfb8be2770dc52c2d39c5eac9d7b9a5a32914ae...,956217002.0,0.059305,0.0,0.0,ACTIVE,NONE,58.0,Dress,Black,Ladieswear,Womens Trend
2020-09-14,cedd533807afaef98ca626a771d8c14986de84bdb43743...,956217002.0,0.059305,0.0,0.0,ACTIVE,NONE,36.0,Dress,Black,Ladieswear,Womens Trend


## 3 Transformación de variables categóricas a numéricas

In [27]:
var_categoricas = ['customer_id', 'club_member_status',"fashion_news_frequency","product_type_name","colour_group_name","index_group_name","section_name"]
encoder = ps.LabelEncoder()

for i in var_categoricas:
    
    sells[i] = encoder.fit_transform(sells[i])

sells.article_id=sells.article_id.astype(int)
sells["FN"]=sells["FN"].astype(int)
sells.Active=sells.Active.astype(int)
sells.age=sells.age.astype(int)
sells=sells.reset_index(drop=True)
print (sells.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5298055 entries, 0 to 5298054
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   customer_id             int32  
 1   article_id              int32  
 2   price                   float64
 3   FN                      int32  
 4   Active                  int32  
 5   club_member_status      int32  
 6   fashion_news_frequency  int32  
 7   age                     int32  
 8   product_type_name       int32  
 9   colour_group_name       int32  
 10  index_group_name        int32  
 11  section_name            int32  
dtypes: float64(1), int32(11)
memory usage: 262.7 MB
None


Separamos directamente los datos de test y de train

In [28]:
sells

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
0,228,108775015,0.008458,0,0,0,1,48,119,1,2,43
1,1322,108775015,0.008458,1,1,0,3,51,119,1,2,43
2,1322,108775015,0.008458,1,1,0,3,51,119,1,2,43
3,1463,108775015,0.008458,0,0,0,1,38,119,1,2,43
4,1870,108775015,0.008458,0,0,2,1,36,119,1,2,43
...,...,...,...,...,...,...,...,...,...,...,...,...
5298050,113026,956217002,0.059305,0,0,0,1,55,31,1,2,53
5298051,212056,956217002,0.059305,1,1,0,3,24,31,1,2,53
5298052,405446,956217002,0.059305,0,0,0,1,50,31,1,2,53
5298053,635275,956217002,0.059305,0,0,0,1,58,31,1,2,53


Para simplificar el problema vamos a cambiar el enfoque del problema, ahora vamos a intentar predecir la recompra de los artículos

In [30]:
sell_sorted = sells.sort_values(['customer_id', 'article_id'])
sells['rebuy'] = sell_sorted['article_id'].duplicated(keep='first')
sells['rebuy'] = sells['rebuy'].astype(int)


In [31]:
sells

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name,rebuy
0,228,108775015,0.008458,0,0,0,1,48,119,1,2,43,0
1,1322,108775015,0.008458,1,1,0,3,51,119,1,2,43,1
2,1322,108775015,0.008458,1,1,0,3,51,119,1,2,43,1
3,1463,108775015,0.008458,0,0,0,1,38,119,1,2,43,1
4,1870,108775015,0.008458,0,0,2,1,36,119,1,2,43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5298050,113026,956217002,0.059305,0,0,0,1,55,31,1,2,53,0
5298051,212056,956217002,0.059305,1,1,0,3,24,31,1,2,53,1
5298052,405446,956217002,0.059305,0,0,0,1,50,31,1,2,53,1
5298053,635275,956217002,0.059305,0,0,0,1,58,31,1,2,53,1


In [33]:
sells.to_csv("data_set.csv",index=False)